# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('../src/')

# Cashflows
## Import Packages

In [ ]:
import pandas as pd
from data.data import flow_data_1, flow_data_2
from interesting.cashflow import Cashflow
from interesting.interest import CompoundInterestRate, YearlyCompoundInterestRate

## Build a Cashflow
### From Json

In [ ]:
cashflow1 = Cashflow().from_json(flow_data_1)
cashflow1.plot_arrow(targets=["brutto"])

### From Pandas DataFrame

In [ ]:
cashflow2 = Cashflow().from_pandas(pd.DataFrame(flow_data_2))
cashflow2.plot_arrow(targets=["brutto"])

# Get information

In [ ]:
print(f"dates: {cashflow1.get_dates()}")
print(f"columns: {cashflow1.get_cols()}")
print(f"point: {cashflow1.get_point(("2026-12-31", "brutto"))}")

## Add Cashflows

In [ ]:
combined_cashflow = cashflow1 + cashflow2

print(combined_cashflow)
combined_cashflow.plot_arrow(targets=["brutto"])

# Interest Rates
## Definition

In [ ]:
r = CompoundInterestRate(0.05, freq="Y")
r_yearly = YearlyCompoundInterestRate(0.05)

print(r == r_yearly)

## Equivalent rates

In [ ]:
# Equivalent rates
r_monthly = CompoundInterestRate(0.015, freq="M")

print((1 + r_monthly.value)**12-1)

r_yearly_equivalent = r_monthly.convert_to_equivalent(new_freq="Y")
print(r_yearly_equivalent)

# Attention!
print(r_yearly_equivalent==r_monthly)

# Tax

In [ ]:
cashflow1.tax()
cashflow1.data

# Valuation

In [ ]:
cashflow1.discount_from_constant_rate(target="netto", interest=r_yearly)
cashflow1.data

In [ ]:
cashflow1.print_npv_and_irr(target="netto")

# Pension

In [ ]:
# How many months will I be in pension? (30 years = 360 months)
n_months = 12*30
# How much money do I want each month?
pension_monthly_salary = 3000
# At which rate is the money pot reinvested?
df = pd.DataFrame()
df["date"] = pd.date_range(start="2025-01-31", periods=n_months+1, freq="ME")
df["brutto"] = [0] + [pension_monthly_salary]*n_months
# market interest rate of 5%/year-> discount rate
r_monthly = YearlyCompoundInterestRate(0.04).convert_to_equivalent(new_freq="M")


# initialize cashflow
pension_cashflow = Cashflow().from_pandas(df)
pension_cashflow = pension_cashflow.discount_from_constant_rate(interest=r_monthly, target="brutto")

# I need 636k to retire with a 3000 salary for 30 years (reinvesting at 4% yearly rate)
pension_cashflow.npv(target="brutto")

In [ ]:
pension_cashflow.data